In [165]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as sa
from sklearn.preprocessing import MinMaxScaler

'''
    FL_DATE is not usefull for regression
    ORIGIN_STATE_ABR is duplicated
    DEST_STATE_ABR is duplicated
    UNIQUE_CARRIER is duplicated of FL_NUM

    Question for Aalap : ??
    1) ORIGIN, ORIGIN_CITY_NAME, ORIGIN_CITY_MARKET_ID
    2) DEST, DEST_CITY_NAME, DEST_CITY_MARKET_ID
       
    I have included all of the three as categorical variables, I think MARKET ID and ORIGIN CITY name
    does not contribute to our cause    
    
    # For printing the nof of values in each features
    # Check the Origin City Market ID and City Values
    for val in categorical_list:
        print(val," : ",df[val].nunique())
      
'''    
# 1) Forming list of attributes headers
drop_list = ['FL_DATE', 'FL_NUM','ORIGIN_STATE_ABR','DEST_STATE_ABR', 'UNIQUE_CARRIER', 'ORIGIN_CITY_MARKET_ID', \
                        'DEST_CITY_MARKET_ID']
    
label_encoding = ['ORIGIN', 'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME']

categorical_list = ['DAY_OF_WEEK','AIRLINE_ID','DISTANCE_GROUP','ORIGIN','ORIGIN_CITY_NAME','DEST','DEST_CITY_NAME']
    
real_list = ['CRS_DEP_TIME', 'TAXI_OUT', 'TAXI_IN', 'ACTUAL_ELAPSED_TIME', 'DISTANCE', 'FIRST_DEP_TIME']


def preprocess(df):     

    df = df.drop('UID', axis = 1)
    # dropping the cols in drop_list
    for val in drop_list:
        df = df.drop(val, axis=1)
    
    # 3) replacing , in non-categorial data represented in the string
    df['DISTANCE'] = df['DISTANCE'].str.replace(',', '')
    df['FIRST_DEP_TIME'] = df['DISTANCE'].str.replace(',','')
    
    # 4) converting the types to numeric & int64
    df[real_list] = df[real_list].apply(pd.to_numeric)
    
    # 5) Scaling the non categorical data
    min_max_scaler = preprocessing.MinMaxScaler()
    df[real_list] = min_max_scaler.fit_transform(df[real_list])
   
    return df
    
def label_and_dummy_endcoding(df):    
    
    # 6) label encoding for city and aiarport codes
    from sklearn.preprocessing import LabelEncoder
    
    labelencoder = LabelEncoder()
    for value in label_encoding:
        df[value] = labelencoder.fit_transform(df[value])
        #print(value)
        #print(df[value])
    
    # 4) converting the types to numeric
    df[categorical_list] = df[categorical_list].astype(int)
    
    # 7) Dummy encoding for all the categorical features
    '''
    One hot encoding is not working, beacuse for it, I have to convert all the literals to int
    
    One difference between both the approaches is: 
    OneHotEncoding does not work with string values of featurs.
    get_dummies() on the contrary works well with strings only
    
    onehotencoder = OneHotEncoder(categorical_features = categorical_list)
    df = onehotencoder.fit_transform(df).toarray()
    '''
    df = pd.get_dummies(df, columns=categorical_list)
    
    return df 

def cal_train_error(predicted, actual):
    return((predicted - actual) ** 2).mean(axis=0)
    
def main():
    
    print(os.getcwd())
    
    # data frame for reading input variables
    train = pd.read_csv('flights_train.csv', header=0).fillna(value = 0)
    test = pd.read_csv('flights_test.csv', header=0)    
    
    # taking the dependent attribute
    delay = (train['ARR_DELAY']).astype(float)    
    train = train.drop('ARR_DELAY', axis = 1)

    #-----------------------------------------
    # Preprocessing and Feature Scaling
    #-----------------------------------------
    
    # preprocessing steps and feature scaling
    train = preprocess(train)
    test = preprocess(test)
    
    #-----------------------------------------
    # Label and Dummy Encoding
    #-----------------------------------------
    
    # for dummy and label encoding
    len_train = len(train)
    train_test = pd.concat([train, test])    
    train_test = label_and_dummy_endcoding(train_test)
    
    train, test = train_test.iloc[:len_train, :], train_test.iloc[len_train:,:]

    #-----------------------------------------
    #Principal Components Analysis
    #-----------------------------------------
    '''
    #perform PCA\
    import sklearn.decomposition as skd
    pca = skd.PCA(n_components =10)
    val=skd.PCA.fit(pca,train)
    W1 = pca.components_
    W = W1.transpose()
    Z = pca.transform(train)
    train = Z
    '''
    
    #---------------------------------
    # prediction algorithms
    #----------------------------------
    
    '''
    Decision Trees
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state = 10)
    regressor.fit(train, delay)
    predicted_val = regressor.predict(train)
    print(cal_train_error(predicted_val, delay))
    '''
    
    '''
    # Random Forests 
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
    regressor.fit(train, delay)
    predicted_val = regressor.predict(train)
        
    print(cal_train_error(predicted_val, delay))
    '''
    
    
    '''
    # Linear Regression
    # Getting very high error on the training set = 1626.7158
    
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(train, delay)
    predicted_val = regressor.predict(train)
    
    print(cal_train_error(predicted_val, delay))
    '''
    
    # KNN algorithm 
    # KNN is slower than Linear Regression
    # for k = 5 error is 1590.17911
    # for k = 10 error is 1794.2212665444993
    from sklearn.neighbors import KNeighborsRegressor as KNNR
    regressor = KNNR(n_neighbors = 10, metric='minkowski', p = 2)
    regressor.fit(train, delay)
    predicted_val = regressor.predict(train)
    print(cal_train_error(predicted_val, delay))
    
    ''' 
    for x in zip(predicted_val, delay):
        print(x)
    '''

if __name__ == '__main__':
    main()


C:\Users\dell
1794.2212665444993
